# Process Media Lists
* Replace raw fields with processed fields

In [ ]:
part = 0

In [ ]:
%run ProcessMediaListsBase.ipynb

In [ ]:
for medium in ["manga", "anime"]:
    process(
        f"../../data/raw_data/user_{medium}_list.pruned.{part}.csv",
        f"../../data/processed_data/user_{medium}_list.{part}.csv",
    )